In [1]:
#Importing usual librairies
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns



In [2]:
# Importing the ML librairies (TO COMPLETE if needed)

from sklearn.preprocessing import MinMaxScaler,StandardScaler,OneHotEncoder
# import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier,AdaBoostClassifier, GradientBoostingClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,mean_absolute_error

In [5]:
# Importing what is needed for API 

import requests

# and MySQL export/import
from sqlalchemy import create_engine 
import pymysql.cursors
import os
# import getpass
import urllib.parse


In [4]:
#Importing the raw data if needed for the original column names
manu1 = pd.read_excel("20221012_IDF_75_all_v2.xls")
#Importing the massaged data from "second look"
df_numbers_light=pd.read_csv("numbers_light.csv")


In [27]:
# creating the url string with the link of the API I want to use

url ="https://api.pappers.fr/v2/entreprise"

# I have saved my password for this API on my config.py file (same folder)
import config

In [29]:
# We need to ask the user for a name of a company, then we get a siren from my other database.
# I use the data for BM EST FRANCE (Groupe Rivalis)

my_siren = "353458086" 

# Defining the request parameters
params = {
    "api_token": config.ACCESS_CODE,  # Assuming you have defined ACCESS_CODE in your config.py file
    "siren": my_siren  # I am calling the SIREN I want query
}

# Connecting to the API, via the GET request with the params
response = requests.get(url, params=params)

# Check the status code, 200 is OK
print(response.status_code)

200


In [30]:
data=response.json()

In [55]:
import csv

In [57]:
# Specify the file path
csv_file_path = 'dictionary.csv'

# Write the dictionary to CSV
with open(csv_file_path, 'w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=data.keys())
    writer.writeheader()
    writer.writerow(data)

In [31]:
data

{'siren': '353458086',
 'siren_formate': '353 458 086',
 'diffusable': True,
 'nom_entreprise': 'BM EST FRANCE',
 'personne_morale': True,
 'denomination': 'BM EST FRANCE',
 'sigle': None,
 'nom': None,
 'prenom': None,
 'sexe': None,
 'siege': {'siret': '35345808600041',
  'siret_formate': '353 458 086 00041',
  'diffusion_partielle': False,
  'nic': '00041',
  'numero_voie': 10,
  'indice_repetition': None,
  'type_voie': None,
  'libelle_voie': "GRAND'RUE",
  'complement_adresse': None,
  'adresse_ligne_1': "10 GRAND'RUE",
  'adresse_ligne_2': None,
  'code_postal': '68280',
  'ville': 'LOGELHEIM',
  'pays': 'France',
  'code_pays': 'FR',
  'latitude': 48.020442,
  'longitude': 7.408359,
  'code_naf': '62.01Z',
  'libelle_code_naf': 'Programmation informatique',
  'date_de_creation': '2006-10-01',
  'etablissement_employeur': True,
  'effectif': 'Entre 50 et 99 salariés',
  'effectif_min': 50,
  'effectif_max': 99,
  'tranche_effectif': '21',
  'annee_effectif': 2021,
  'etablisseme

In [33]:
#Analysing the data from the API
type(data)
data.keys()

dict_keys(['siren', 'siren_formate', 'diffusable', 'nom_entreprise', 'personne_morale', 'denomination', 'sigle', 'nom', 'prenom', 'sexe', 'siege', 'rnm', 'code_naf', 'libelle_code_naf', 'domaine_activite', 'objet_social', 'conventions_collectives', 'date_creation', 'date_creation_formate', 'entreprise_cessee', 'date_cessation', 'date_cessation_formate', 'associe_unique', 'categorie_juridique', 'forme_juridique', 'forme_exercice', 'entreprise_employeuse', 'societe_a_mission', 'effectif', 'effectif_min', 'effectif_max', 'annee_effectif', 'tranche_effectif', 'annee_tranche_effectif', 'capital', 'capital_actuel_si_variable', 'devise_capital', 'capital_formate', 'date_cloture_exercice', 'date_cloture_exercice_exceptionnelle', 'prochaine_date_cloture_exercice', 'prochaine_date_cloture_exercice_formate', 'economie_sociale_solidaire', 'duree_personne_morale', 'derniere_mise_a_jour_sirene', 'derniere_mise_a_jour_rcs', 'derniere_mise_a_jour_rne', 'dernier_traitement', 'date_debut_activite', 'dat

In [96]:
data['representants'][0]['personne_morale']
data['representants'][0]['nom_complet']

'PHOSPHORE'

In [109]:
# elements I need to create the 1 line 4x dataframes; TO USE for the names of the columns
# General data
general = ["name", "siren", "post_code", "city", "naf2_code", "naf2_activity"]

row_general = [data['nom_entreprise'],data['siren'],data['siege']['code_postal'], data['siege']['ville'],data['code_naf'],
              data['libelle_code_naf']+'. '+ data['domaine_activite']]


In [110]:
# function to assign the group for size

def size_group(x):
    limits =[1, 9, 15, 22, 31, 41, 52, 79, 131, 269]
    if x:
        # will create a loop if time...
        if limits[0] <= x < limits[1]:
            return 'q1'
        elif limits[1] <= x < limits[2]:
            return 'q2'
        elif limits[2] <= x < limits[3]:
            return 'q3'
        elif limits[3] <= x < limits[4]:
            return 'q4'       
        elif limits[4] <= x < limits[5]:
            return 'q5'
        elif limits[5] <= x < limits[6]:
            return 'q6'
        elif limits[6] <= x < limits[7]:
            return 'q7'
        elif limits[7] <= x < limits[8]:
            return 'q8'
        elif limits[8] <= x < limits[9]:
            return 'q9'       
        elif limits[9] <= x:
            return 'q10'
        else:
            return None
    else:
        return None


In [ ]:
# ... and specific values calculated

# Calculation of avg_workforce:
workforce = round((data['effectif_max']-data['effectif_min'])/2)

# Revenue
revenue = data['finances'][0]['chiffre_affaires']

# Operating income
opera_income = data['finances'][0]['resultat_exploitation']

# Result
result = data['finances'][0]['resultat']

# Depreciation estimation... => EBIDTA - result - taxes - exceptional - financial (works ONLY if exceptional is low)
depreciation_est = opera_income*data['finances'][0]['taux_marge_EBITDA'] - result - data['finances'][0]['impots_taxes']

# Borrowed
borrowed = data['finances'][0]['dettes_financieres'] + data['finances'][0]['etat_dettes_1_an_au_plus']

# Personne morale: natural_or_legal, True for "Entreprise", False for "Personne"
if data['representants'][0]['personne_morale'] != None:
    if data['representants'][0]['personne_morale'] == True:
        nature = "Entreprise"
    else:
        nature = "Personne"
else:
    nature = None

In [112]:
# Numbers light data
numbers_light =["name", "siren","naf2_code","avg_workforce","size_group","revenue_10m","op_inc_percent","dep_percent","borrowed", 
                'rentals', 'cash_percent', "secure_investment_percent"]

row_numbers_light = [data['nom_entreprise'],data['siren'], data['code_naf'], workforce, size_group(workforce), revenue/10000000,
                    opera_income/revenue, depreciation_est/revenue, borrowed/revenue, None, data['finances'][0]['tresorerie']/revenue,
                    None]

In [114]:
# Numbers_all data
numbers_all = ["name", "siren","avg_workforce",  "revenue", "depre_amort", "operating_income", "convert_bonds", "other_bonds", "loans_less_1yr", 
           "loans_more_1yr", "other_loans", "debts_to_group", "commit_mov_prop", "commit_immov_prop", "bills_exch", "cash",
           "market_security"]

row_numbers_all = [data['nom_entreprise'],data['siren'], workforce, revenue/1000, depreciation_est/1000 ,opera_income/1000,
                  0, data['finances'][0]['dettes_financieres']/1000, 0, data['finances'][0]['etat_dettes_1_an_au_plus']/1000,
                  0, 0, None, None, None, data['finances'][0]['tresorerie']/1000, 0 ]

#Management data
management = ["name", "siren","ceo", "natural_or_legal", "date_of_birth", "group_head", "shareholder", "shareholder_1stname", 
              "group_head_1stname", "group_head_share","shareholder_share", "reference_shareholder", 
              "reference_shareholder_1stname", "reference_shareholder_share"]

row_management = [data['nom_entreprise'],data['siren'],data['representants'][0]['nom_complet'], nature, None, None, None, None,
                 None, None, None, None, None, None]

In [115]:
len(management), len(row_management)

(14, 14)

In [116]:
# creating the 4 dataframes for the request of the user with the data from the API

small_general = pd.DataFrame([row_general], columns = general)
small_numbers_all = pd.DataFrame([row_numbers_all], columns = numbers_all)
small_management = pd.DataFrame([row_management], columns = management)
small_numbers_light = pd.DataFrame([row_numbers_light], columns = numbers_light)


In [118]:
display(small_general)
display(small_numbers_all)
display(small_management)
display(small_numbers_light)

,name,siren,post_code,city,naf2_code,naf2_activity
0,BM EST FRANCE,353458086,68280,LOGELHEIM,82.99Z,Autres activités de soutien aux entreprises n....


,name,siren,avg_workforce,revenue,depre_amort,operating_income,convert_bonds,other_bonds,loans_less_1yr,loans_more_1yr,other_loans,debts_to_group,commit_mov_prop,commit_immov_prop,bills_exch,cash,market_security
0,BM EST FRANCE,353458086,24,10519.818,-6570.0598,889.657,0,103.728,0,8742.835,0,0,None,None,None,790.319,0


,name,siren,ceo,natural_or_legal,date_of_birth,group_head,shareholder,shareholder_1stname,group_head_1stname,group_head_share,shareholder_share,reference_shareholder,reference_shareholder_1stname,reference_shareholder_share
0,BM EST FRANCE,353458086,PHOSPHORE,Entreprise,None,None,None,None,None,None,None,None,None,None


,name,siren,naf2_code,avg_workforce,size_group,revenue_10m,op_inc_percent,dep_percent,borrowed,rentals,cash_percent,secure_investment_percent
0,BM EST FRANCE,353458086,82.99Z,24,q4,1.051982,0.08457,-0.624541,0.840943,None,0.075127,None


In [119]:
# generating usable dataframes from API

def conversion(data, size_group):
    """Converts the data -in dictionary format- from the API and using the function size_group to
    into a list of 4 usable dataframes """
    
    # specific values calculated

        # Calculation of avg_workforce:
    workforce = round((data['effectif_max']-data['effectif_min'])/2)

        # Revenue
    revenue = data['finances'][0]['chiffre_affaires']

        # Operating income
    opera_income = data['finances'][0]['resultat_exploitation']

        # Result
    result = data['finances'][0]['resultat']

        # Depreciation estimation... => EBIDTA - result - taxes - exceptional - financial (works ONLY if exceptional is low)
    depreciation_est = opera_income*data['finances'][0]['taux_marge_EBITDA'] - result - data['finances'][0]['impots_taxes']

        # Borrowed
    borrowed = data['finances'][0]['dettes_financieres'] + data['finances'][0]['etat_dettes_1_an_au_plus']

        # Personne morale: natural_or_legal, True for "Entreprise", False for "Personne"
    if data['representants'][0]['personne_morale'] != None:
        if data['representants'][0]['personne_morale'] == True:
            nature = "Entreprise"
        else:
            nature = "Personne"
    else:
        nature = None
    
    # Creating the first dataframe
    general = ["name", "siren", "post_code", "city", "naf2_code", "naf2_activity"]

    row_general = [data['nom_entreprise'],data['siren'],data['siege']['code_postal'], data['siege']['ville'],data['code_naf'],
              data['libelle_code_naf']+'. '+ data['domaine_activite']]
    
    small_general = pd.DataFrame([row_general], columns = general)
    
    # Creating the second dataframe
    # Numbers light data
    numbers_light =["name", "siren","naf2_code","avg_workforce","size_group","revenue_10m","op_inc_percent","dep_percent",
                    "borrowed", 'rentals', 'cash_percent', "secure_investment_percent"]

    row_numbers_light = [data['nom_entreprise'],data['siren'], data['code_naf'], workforce, size_group(workforce), 
                         revenue/10000000, opera_income/revenue, depreciation_est/revenue, borrowed/revenue, None, 
                         data['finances'][0]['tresorerie']/revenue, None]    
    
    small_numbers_light = pd.DataFrame([row_numbers_light], columns = numbers_light)
    
    # Creating the third dataframe
    # Numbers_all data
    numbers_all = ["name", "siren","avg_workforce",  "revenue", "depre_amort", "operating_income", "convert_bonds", "other_bonds",
                   "loans_less_1yr", "loans_more_1yr", "other_loans", "debts_to_group", "commit_mov_prop", "commit_immov_prop",
                   "bills_exch", "cash", "market_security"]

    row_numbers_all = [data['nom_entreprise'],data['siren'], workforce, revenue/1000, depreciation_est/1000 ,opera_income/1000,
                       0, data['finances'][0]['dettes_financieres']/1000, 0, data['finances'][0]['etat_dettes_1_an_au_plus']/1000,
                       0, 0, None, None, None, data['finances'][0]['tresorerie']/1000, 0 ]
    
    small_numbers_all = pd.DataFrame([row_numbers_all], columns = numbers_all)

    # Creating the fourth dataframe
    #Management data
    management = ["name", "siren","ceo", "natural_or_legal", "date_of_birth", "group_head", "shareholder", "shareholder_1stname",
                  "group_head_1stname", "group_head_share","shareholder_share", "reference_shareholder", 
                  "reference_shareholder_1stname", "reference_shareholder_share"]

    row_management = [data['nom_entreprise'],data['siren'],data['representants'][0]['nom_complet'], nature, None, None, None, 
                      None, None, None, None, None, None, None]    
    
    small_management = pd.DataFrame([row_management], columns = management)

    
    return [small_general, small_numbers_light, small_numbers_all, small_management]
    
    

In [ ]:
# TO DO concat the data to the 4 main tables if siren did not exist. else replace the non null values?
# to use the small_numbers_light for the model (using numbers_light)